## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
import keras

Using TensorFlow backend.
C:\Users\wilson\Anaconda3\envs\tf-wilson\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\wilson\Anaconda3\envs\tf-wilson\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\wilson\Anaconda3\envs\tf-wilson\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\wilson\Anaconda3\envs\t

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(y_test[5]) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

[6]


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()
print(y_train.shape)
print(y_test.shape)

# y_train = keras.utils.to_categorical(y_train, 10)
# y_test = keras.utils.to_categorical(y_test, 10)
# print(y_train.shape)
# print(y_test.shape)

(50000, 10)
(10000, 10)


In [4]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
 
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
 
classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32,3,3,input_shape=(32,32,3),activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
#classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(64,3,3,activation='relu'))
classifier.add(BatchNormalization())
#classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(Convolution2D(128,3,3,activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))
#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(output_dim=1024,activation='relu')) #output_dim=100,activation=relu
classifier.add(BatchNormalization())
#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))
classifier.summary()
#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=10)

C:\Users\wilson\Anaconda3\envs\tf-wilson\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(32, 32, 3..., activation="relu")`
  # This is added back by InteractiveShellApp.init_path()
C:\Users\wilson\Anaconda3\envs\tf-wilson\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
C:\Users\wilson\Anaconda3\envs\tf-wilson\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
C:\Users\wilson\Anaconda3\envs\tf-wilson\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=1024)`


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 28, 28, 64)        256       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 26, 128)       73856     
_________________________________________________________________
batch_normalization_3 (Batch (None, 26, 26, 128)       512       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 128)      

C:\Users\wilson\Anaconda3\envs\tf-wilson\lib\site-packages\ipykernel_launcher.py:32: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`



Epoch 1/10
50000/50000 [==============================] - 14s 275us/step - loss: 1.2764 - accuracy: 0.5781
Epoch 2/10
50000/50000 [==============================] - 12s 231us/step - loss: 0.7866 - accuracy: 0.7272
Epoch 3/10
50000/50000 [==============================] - 12s 233us/step - loss: 0.5447 - accuracy: 0.8119
Epoch 4/10
50000/50000 [==============================] - 12s 239us/step - loss: 0.3076 - accuracy: 0.8956s - loss: 0
Epoch 5/10
50000/50000 [==============================] - 12s 242us/step - loss: 0.1218 - accuracy: 0.9625
Epoch 6/10
50000/50000 [==============================] - 12s 232us/step - loss: 0.0686 - accuracy: 0.9801
Epoch 7/10
50000/50000 [==============================] - 12s 233us/step - loss: 0.0662 - accuracy: 0.9792
Epoch 8/10
50000/50000 [==============================] - 12s 232us/step - loss: 0.0638 - accuracy: 0.9794
Epoch 9/10
50000/50000 [==============================] - 12s 235us/step - loss: 0.0597 - accuracy: 0.9817
Epoch 10/10
50000/50000 [

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [7]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

NameError: name 'mean_train' is not defined